In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [2]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
lending_DataDF = pd.read_csv('lending_data.csv')

# Review the DataFrame
lending_DataDF.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [3]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = lending_DataDF['loan_status']

# Separate the X variable, the features
X = lending_DataDF.drop(columns=['loan_status'])

In [4]:
# Review the y variable Series
y.head()

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [5]:
# Review the X variable DataFrame
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [6]:
# Check the balance of our target values
y.value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [7]:
# Import the train_test_split module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [8]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
logistic_RegressionModel = LogisticRegression(random_state=1)

# Fit the model using training data
logistic_RegressionModel.fit(X_train, y_train)

LogisticRegression(random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [9]:
# Make a prediction using the testing data
testing_Predictions = logistic_RegressionModel.predict(X_test)
print(len(testing_Predictions))

19384


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the balanced accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [10]:
# Print the balanced_accuracy score of the model
balanced_AccuracyScoreLR = balanced_accuracy_score(y_test, testing_Predictions)
print(f"Balanced Accuracy Score: {balanced_AccuracyScoreLR}")

Balanced Accuracy Score: 0.9442676901753825


In [11]:
# Generate a confusion matrix for the model
logistic_matrix = confusion_matrix(y_test, testing_Predictions)
print(logistic_matrix)

[[18679    80]
 [   67   558]]


In [12]:
# Print the classification report for the model
classification_Report = classification_report(y_test, testing_Predictions)

# View the result
print(f'The classification report with the original data shows:\n{classification_Report}')

The classification report with the original data shows:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18759
           1       0.87      0.89      0.88       625

    accuracy                           0.99     19384
   macro avg       0.94      0.94      0.94     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** For healthy loans the precision is 100%, the recall is 100%, and the f1-score is 100%, which means that the model does really well predicting healthy loan instances.

For high-risk loans the precision is 87%, the recall is 89%, and the f1-score is 88%, which means that the model does moderately well in predicting high-risk loan instances.

Overall, the balanced accuracy of the model is 94%, which indicates how well the model is performing across both classes of loans.

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points.

In [14]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# Assign a random_state parameter of 1 to the model
randomOverSamplerModel = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
x_res, y_res = randomOverSamplerModel.fit_resample(X_train, y_train)

In [15]:
# Count the distinct values of the resampled labels data
y_res.value_counts()

0    56277
1    56277
Name: loan_status, dtype: int64

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [16]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using the resampled training data
classifier.fit(x_res, y_res)

# Make a prediction using the testing data
res_Predictions = classifier.predict(X_test)
pd.DataFrame({'Prediction': res_Predictions, 'Actual': y_test})

,Prediction,Actual
36831,0,0
75818,1,1
36563,0,0
13237,0,0
43292,0,0
...,...,...
38069,0,0
36892,0,0
5035,0,0
40821,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the balanced accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [17]:
# Print the balanced_accuracy score of the model
balanced_AccuracyScoreROS = balanced_accuracy_score(y_test, res_Predictions)
print(f"Balanced Accuracy Score: {balanced_AccuracyScoreROS}")

Balanced Accuracy Score: 0.9959744975744975


In [18]:
# Generate a confusion matrix for the model
ros_matrix = confusion_matrix(y_test, res_Predictions)
print(ros_matrix)

[[18668    91]
 [    2   623]]


In [19]:
# Print the classification report for the model
classification_ReportROS = classification_report(y_test, res_Predictions)

# View the result
print(classification_ReportROS)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18759
           1       0.87      1.00      0.93       625

    accuracy                           1.00     19384
   macro avg       0.94      1.00      0.96     19384
weighted avg       1.00      1.00      1.00     19384



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The logistic regression model with resampled training data predicts a healthy (i.e., low-risk) loan with a precision of 100%, recall of 100%, and f1-score of 100%.

Additionally, the model predicts a high-risk loan with a moderately lower precision of 87%, recall of 100%, and f1-score of 93%, but performs incrementally better before oversampled data.

Overall, the balanced accuracy of the model is 100%, which indicates perfect recall across both classes with no bias towards the majority or minority classes.